In [ ]:
from selenium import webdriver
import pandas as pd
import csv
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup


In [ ]:
url = 'https://www.thegioididong.com/laptop-asus#c=44&m=122,128,119,120,118,203&o=17&pi=12' 
s = Service('E:\project\crawl-data\script\chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get(url)

In [ ]:
page_source = BeautifulSoup(driver.page_source)

In [ ]:
laptop_list = page_source.find_all('a',attrs={'class':'main-contain', 'data-box':'BoxCateFilter'})
link_list = []
for laptop in laptop_list:
    laptop_link = laptop.get('href')
    if laptop_link not in link_list:
        link_list.append('https://www.thegioididong.com' + laptop_link)
link_list
len(link_list)

In [18]:
names  = []
old_prices = []
present_prices = []
discounts = []
due_discounts = []
types = []   # Loại thiết bị
trademarks = [] # Thương hiệu
series = []    # Dòng sản phẩm
links = link_list

In [19]:
for link in link_list[100:150]:
    driver.get(link)
    sleep(10)
    page_source = BeautifulSoup(driver.page_source)
    try:
        #Lấy tên sản phẩm
        name = page_source.find('section', attrs={'data-cate-id':'44','class':'detail'}).find('h1').get_text().strip()
        
        #Lấy giá gốc của sản phẩm
        old_price = page_source.find('p',class_='box-price-old').get_text().strip() 
        
        #Lấy giá đã giảm của sản phẩm
        present_price = page_source.find('p',class_='box-price-present').get_text().strip() 
        
        #Lấy phần trăm discount của sản phẩm
        percent = page_source.find('p',class_='box-price-percent')
        discount = 'empty'
        if percent == None :
            discount = 'empty'
        else:
            discount = percent.get_text().strip()
            
        #Lấy hạn giảm giá
        due_discount = ' '.join(page_source.find('div', class_='pr-top').find('i',class_='pr-txt').get_text().strip().split(' ')[-3:])
        
        #Lấy loại thiết bị
        type = page_source.find('section', attrs={'data-cate-id':'44','class':'detail'}).find_all('li')[0].get_text().strip()
        
        #Lấy tên thương hiệu
        trademark = page_source.find('section', attrs={'data-cate-id':'44','class':'detail'}).find_all('li')[1].get_text().strip().split(' ')[1]
        
        #Lấy dòng sản phẩm
        temp = name.split(' ')
        serie = temp[temp.index(trademark) + 1]
        
        
        #Thêm các giá trị vào list tương ứng nếu không có lỗi
        names.append(name)
        old_prices.append(old_price)
        present_prices.append(present_price)
        discounts.append(discounts)
        due_discounts.append(due_discount)
        types.append(type)
        trademarks.append(trademark)
        series.append(serie)
        
    except :
        pass

In [17]:
data = {'name' : names,\
        'old_price' : old_prices,\
        'present_price' : present_prices,\
        'discount' : discounts,\
        'due_discount' : due_discounts,\
        'type_of_device' : types,\
        'trademark' : trademarks,\
        'series' : series,\
        'link' : links[100:150]
        }
df = pd.DataFrame(data)
df


,name,old_price,present_price,discount,due_discount,type_of_device,trademark,series,link
0,Laptop Asus Gaming TUF Dash F15 FX517ZE i5 124...,24.990.000₫,20.490.000₫ *,"[[[[[...], [...], [...], [...], [...]], [[...]...",23:00 | 30/11,Laptop,Asus,Gaming,https://www.thegioididong.com/laptop/asus-tuf-...
1,Laptop Asus Gaming ROG Strix G15 G513IH R7 480...,22.990.000₫,17.990.000₫ *,"[[[[[...], [...], [...], [...], [...]], [[...]...",23:00 | 30/11,Laptop,Asus,Gaming,https://www.thegioididong.com/laptop/asus-rog-...
2,Laptop Asus Vivobook 14X OLED A1405ZA i5 12500...,19.490.000₫,16.990.000₫ *,"[[[[[...], [...], [...], [...], [...]], [[...]...",23:00 | 30/11,Laptop,Asus,Vivobook,https://www.thegioididong.com/laptop/asus-vivo...
3,Laptop Acer Aspire Vero AV15 51 58HB i5 1155G7...,17.990.000₫,12.990.000₫ *,"[[[[[...], [...], [...], [...], [...]], [[...]...",23:00 | 30/11,Laptop,Acer,Aspire,https://www.thegioididong.com/laptop/acer-aspi...
4,Laptop Acer Nitro 5 Tiger AN515 58 52SP i5 125...,27.990.000₫,22.990.000₫ *,"[[[[[...], [...], [...], [...], [...]], [[...]...",23:00 | 30/11,Laptop,Acer,Nitro,https://www.thegioididong.com/laptop/acer-nitr...


In [14]:
names

[]

In [ ]:
df.to_csv('E:\project\crawl-data\outputed-csv-file\laptop_tgdd_100_end.csv', mode = 'a')